In [4]:
import sys
sys.path.append('/Users/qiaoli/Desktop/PVZ/PvZ-Cobless-Simulator-main/new')
from info import *

In [ ]:
    
test_N = 1 * 10**5   
       
slow_t = []         
ice_t = []

M = 2000
N = test_N

In [6]:
def generate_animation_x(N_animation, total_x, x_each_animation, v_range=(0.66,0.68), x_range=(780, 820)):
    v = np.random.uniform(v_range[0],v_range[1], size=N)
    v = np.linspace(v_range[0], v_range[1], N) # 测试用

    '''
    # jack_type 全局变量
    explode_t = np.zeros(N, dtype=int)
    if jack_type == 1:
        explode_t = 2 * (((np.random.randint(0, 300, size=N) + 450) / 3.0).astype(int) / v).astype(int)
    elif jack_type == 2:
        explode_t = 2 * (((np.random.randint(0, 300, size=N) + 450) / v).astype(int))
    else:
        mask_early = (np.random.randint(0, 20, size=N)==0)
        if mask_early.any():
            explode_t[mask_early] = 2 * (((np.random.randint(0, 300, size=mask_early.sum()) + 450) / 3.0).astype(int) / v[mask_early]).astype(int)
        if (~mask_early).any():
            explode_t[~mask_early] = 2 * (((np.random.randint(0, 300, size=(~mask_early).sum()) + 450) / v[~mask_early]).astype(int))
    '''

    s = np.full(shape=(N,M),fill_value=2,dtype=int) # tcs末的状态
    
    for t in slow_t:    s[:, min(t+1):min(t+1000,M)] = 1
    
    frozen_times = np.zeros(N, dtype=int)
    for t in ice_t:
        mask_normal = (s[:, t-1] == 2) & (s[:, t] == 2)
        if mask_normal.any():  
            frozen_times[mask_normal] = np.random.randint(399, 600, size=mask_normal.sum())
        if (~mask_normal).any():  
            frozen_times[~mask_normal] = np.random.randint(299, 400, size=(~mask_normal).sum())
        
        '''
        mask_delay = (t <= explode_t)
        explode_t[mask_delay] += frozen_times[mask_delay] + 1
        '''

        frozen_end_times = np.clip(t + frozen_times, 0 ,M)
        slowdown_end_times = min(t+1999, M)
        for j in range(N):
            s[j, t:frozen_end_times[j]] = 0
            s[j, frozen_end_times[j]:slowdown_end_times] = 1
    del frozen_times
    
    speed = s*v[:, np.newaxis]/2.0  
    del s
    
    delta_completing_rate = 47*0.00999999977648258 /total_x*speed
    delta_completing_rate[:, 0] = 47*0.00999999977648258/total_x * v
    del v
    delta_completing_rate = np.concatenate([np.zeros((N, 1)), delta_completing_rate[:, :-1]], axis=1)

    cum_completing_rate = np.cumsum(delta_completing_rate, axis=1)
    del delta_completing_rate
    completing_rate, _ = np.modf(cum_completing_rate)
    del cum_completing_rate
    
    x = - 47*0.009999999776482582 / total_x*(1+N_animation)* speed * x_each_animation[(completing_rate*N_animation).astype(int)]
    del completing_rate, speed
    
    x[:,0] = np.random.randint(x_range[0], x_range[1], size=N) 
    x = np.cumsum(x, axis=1)
    return x #, explode_t

In [7]:
def generate_zomboni_x():
    x = np.zeros(M)
    x[0] = 800.0
    for i in range(1,M):
        tmp = x[i-1].astype(int)
        if tmp>700:     dlt_x = 0.25
        elif tmp<=400:  dlt_x = 0.10
        else:           dlt_x = 0.0005*tmp - 0.10
        x[i] = x[i-1] - dlt_x
    return x

In [8]:
x = generate_animation_x(*ANIMATION_ZOMBIE_INFO['jack'].get_info()[:3],
                         v_range=(0.66,0.68), x_range=(780, 781))

In [9]:
pd.DataFrame({
    't': np.arange(0, M, dtype=np.int16),
    'min_x': np.round(x.min(axis=0), decimals=3),
    'max_x': np.round(x.max(axis=0), decimals=3)
}).to_csv('x_64.csv', index=False)